In [1]:
import graphlab as gl
import pandas as pd

In [2]:
titanic_sf = gl.SFrame.read_csv('train.csv')
titanic_test_sf = gl.SFrame.read_csv('test.csv')

This non-commercial license of GraphLab Create for academic use is assigned to phanikiran99@gmail.com and will expire on September 06, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\IBM_AD~1\AppData\Local\Temp\graphlab_server_1474353120.log.0


Finished parsing file C:\Users\IBM_ADMIN\jupynb\kaggle\Titanic\train.csv

Parsing completed. Parsed 100 lines in 0.140008 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long,str,str,float,long,long,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Finished parsing file C:\Users\IBM_ADMIN\jupynb\kaggle\Titanic\test.csv

Parsing completed. Parsed 100 lines in 0.014001 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,str,str,float,long,long,str,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\IBM_ADMIN\jupynb\kaggle\Titanic\test.csv

Parsing completed. Parsed 418 lines in 0.015001 secs.

In [3]:
titanic_sf.head(5)

Columns:
	PassengerId	int
	Survived	int
	Pclass	int
	Name	str
	Sex	str
	Age	float
	SibSp	int
	Parch	int
	Ticket	str
	Fare	float
	Cabin	str
	Embarked	str

Rows: 5

Data:
+-------------+----------+--------+-------------------------------+--------+
| PassengerId | Survived | Pclass |              Name             |  Sex   |
+-------------+----------+--------+-------------------------------+--------+
|      1      |    0     |   3    |    Braund, Mr. Owen Harris    |  male  |
|      2      |    1     |   1    | Cumings, Mrs. John Bradley... | female |
|      3      |    1     |   3    |     Heikkinen, Miss. Laina    | female |
|      4      |    1     |   1    | Futrelle, Mrs. Jacques Hea... | female |
|      5      |    0     |   3    |    Allen, Mr. William Henry   |  male  |
+-------------+----------+--------+-------------------------------+--------+
+------+-------+-------+------------------+---------+-------+----------+
| Age  | SibSp | Parch |      Ticket      |   Fare  | Cabin | Embarked |
+------+-------+-------+------------------+---------+-------+----------+
| 22.0 |   1   |   0   |    A/5 21171     |   7.25  |       |    S     |
| 38.0 |   1   |   0   |     PC 17599     | 71.2833 |  C85  |    C     |
| 26.0 |   0   |   0   | STON/O2. 3101282 |  7.925  |       |    S     |
| 35.0 |   1   |   0   |      113803      |   53.1  |  C123 |    S     |
| 35.0 |   0   |   0   |      373450      |   8.05  |       |    S     |
+------+-------+-------+------------------+---------+-------+----------+
[5 rows x 12 columns]

In [4]:
gl.canvas.set_target('ipynb') #Setting canvas target to notebook

In [5]:
titanic_sf['Survived'].show(view='Categorical')

In [6]:
# Let us clean the data 
def clean_up_df(df):
    """ This function will cleanup Age(Median), Sex(Change to 0,1), SibSp,Parch(Merge to Family), Embarked data
    Update to 'S' And Also deletes Name,Cabin details from titanic DF, Ensure to Pass DataFrame to this Function"""
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Gender'] = df['Sex'].map({'female':0, 'male':1}).astype(int)
    df['Family'] = df['Parch'] + df['SibSp']
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    df = df.drop(['SibSp','Parch','Sex','Name','Cabin','Embarked','Ticket'],axis=1)
    return df
titanic_df = titanic_sf.to_dataframe()
titanic_test_df = titanic_test_sf.to_dataframe()
cleaned_sf = gl.SFrame(clean_up_df(titanic_df))
test_sf = gl.SFrame(clean_up_df(titanic_test_df))

In [7]:
type(test_sf)

graphlab.data_structures.sframe.SFrame

In [8]:
cleaned_sf['Pclass','Age','Survived'].show()

In [9]:
cleaned_sf.head(2)

PassengerId,Survived,Pclass,Age,Fare,Gender,Family
1,0,3,22.0,7.25,1,1
2,1,1,38.0,71.2833,0,1


In [10]:
# Train logistic classifier
train_data,test_data = cleaned_sf.random_split(0.9,seed=0)
logistic_model = gl.logistic_classifier.create(cleaned_sf,target='Survived',features=['Pclass','Age','Fare','Gender','Family'],
                                              validation_set=test_data,max_iterations=20)

Logistic regression:

--------------------------------------------------------

Number of examples          : 891

Number of classes           : 2

Number of feature columns   : 5

...

| 2         | 3        | 1.117064     | 0.799102          | 0.744681            |

| 3         | 4        | 1.126065     | 0.799102          | 0.734043            |

| 4         | 5        | 1.134065     | 0.800224          | 0.734043            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [11]:
logistic_model.evaluate(test_data,metric='roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+----+----+
 | threshold | fpr | tpr | p  | n  |
 +-----------+-----+-----+----+----+
 |    0.0    | 1.0 | 1.0 | 35 | 59 |
 |   1e-05   | 1.0 | 1.0 | 35 | 59 |
 |   2e-05   | 1.0 | 1.0 | 35 | 59 |
 |   3e-05   | 1.0 | 1.0 | 35 | 59 |
 |   4e-05   | 1.0 | 1.0 | 35 | 59 |
 |   5e-05   | 1.0 | 1.0 | 35 | 59 |
 |   6e-05   | 1.0 | 1.0 | 35 | 59 |
 |   7e-05   | 1.0 | 1.0 | 35 | 59 |
 |   8e-05   | 1.0 | 1.0 | 35 | 59 |
 |   9e-05   | 1.0 | 1.0 | 35 | 59 |
 +-----------+-----+-----+----+----+
 [100001 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.}

In [12]:
logistic_model.show(view='Evaluation')

In [13]:
test_sf['Survived'] = logistic_model.predict(test_sf)

In [14]:
test_sf

PassengerId,Pclass,Age,Fare,Gender,Family,Survived
892,3,34.5,7.8292,1,0,0
893,3,47.0,7.0,0,1,0
894,2,62.0,9.6875,1,0,0
895,3,27.0,8.6625,1,0,0
896,3,22.0,12.2875,0,2,1
897,3,14.0,9.225,1,0,0
898,3,30.0,7.6292,0,0,1
899,2,26.0,29.0,1,2,0
900,3,18.0,7.2292,0,0,1
901,3,21.0,24.15,1,2,0


In [15]:
submission = pd.DataFrame({
        'Passengerid':test_sf['PassengerId'],
        'Survived':test_sf['Survived']
    })
submission.to_csv('titanic_gl.csv')

In [16]:
# This submission got a score of 75% which is not a major improvement though. 